# Chapter 12: Custom Models and Training with TensorFlow

Meskipun API tingkat tinggi `tf.keras` sudah mencakup 95% kasus penggunaan, terkadang kita membutuhkan kendali ekstra. Bab ini menyelami API tingkat rendah TensorFlow untuk membuat komponen kustom seperti fungsi aktivasi, regularizer, lapisan, hingga model yang sangat kompleks.

## Konten Utama:
1. **TensorFlow Dasar**: Manipulasi Tensor dan Variabel.
2. **Kustomisasi Komponen Keras**: Membuat fungsi loss, metrik, dan layer buatan sendiri.
3. **Autodiff & Gradient Tape**: Mekanisme di balik perhitungan gradien otomatis.
4. **Custom Training Loops**: Menjalankan proses pelatihan secara manual tanpa `.fit()`.
5. **TF Functions & Graphs**: Mengoptimalkan performa kode Python menjadi grafik komputasi yang efisien.

## 1. Menggunakan TensorFlow seperti NumPy

Tensor adalah inti dari TensorFlow. Mirip dengan array NumPy, tetapi mendukung akselerasi GPU dan operasi terdistribusi.

In [ ]:
import tensorflow as tf
import numpy as np

# Membuat Tensor
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print("Tensor:\n", t)

# Operasi Dasar
print("Tambah 10:\n", t + 10)
print("Square:\n", tf.square(t))

# Variabel (Mutable - digunakan untuk bobot model)
v = tf.Variable([[1., 2.], [3., 4.]])
v.assign(2 * v)
v[0, 1].assign(42)
print("Variable:\n", v)

## 2. Custom Loss Functions

Misalkan kita ingin menggunakan **Huber Loss** yang tidak tersedia secara standar atau ingin dikustomisasi threshold-nya.

In [ ]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# Penggunaan dalam Model
# model.compile(loss=huber_fn, optimizer="nadam")

## 3. Custom Layers & Models

Kita bisa membuat lapisan kustom dengan melakukan subclassing pada `keras.layers.Layer`.

In [ ]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        # Membuat bobot (weights)
        self.kernel = self.add_weight(name="kernel", 
                                      shape=[batch_input_shape[-1], self.units],
                                      initializer="glorot_normal")
        self.bias = self.add_weight(name="bias", shape=[self.units], 
                                    initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

# Menggunakan layer kustom dalam model
model = tf.keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=[8]),
    MyDense(1)
])

## 4. Perhitungan Gradien dengan Autodiff

TensorFlow menggunakan `tf.GradientTape` untuk merekam operasi komputasi sehingga gradien dapat dihitung secara otomatis.

In [ ]:
def f(w1, w2):
    return 3 * w1**2 + 2 * w1 * w2

w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
print("Gradients [dz/dw1, dz/dw2]:", gradients)

## 5. Custom Training Loops

Untuk kendali penuh atas proses update bobot, kita bisa menulis loop pelatihan sendiri tanpa menggunakan `.fit()`.

In [ ]:
# Skenario sederhana Custom Training Loop
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error

def train_step(model, X_batch, y_batch):
    with tf.GradientTape() as tape:
        y_pred = model(X_batch)
        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

## 6. TF Functions & Graphs

Gunakan dekorator `@tf.function` untuk mengubah fungsi Python biasa menjadi grafik TensorFlow yang sangat cepat.

In [ ]:
@tf.function
def tf_cube(x):
    return x ** 3

print("Cube of 2:", tf_cube(tf.constant(2.0)))

## Kesimpulan Bab 12
Bab ini memberikan dasar bagi pengembang Deep Learning untuk tidak hanya menjadi 'pengguna' library, tetapi juga 'pencipta' arsitektur baru. Dengan memahami Tensor, GradientTape, dan subclassing, Anda siap untuk mengimplementasikan paper penelitian terbaru ke dalam kode.